In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

from IPython.display import clear_output
from datetime import datetime

In [2]:
param_date = '20240726'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel(f'./{param_date}/1.manufacturer.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Manufacturer No'] = input_['Manufacturer No'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        import requests
        import json

        import sys
        sys.path.append('../../00.Tools')
        from crawler_configuration import get_header, get_proxy

        data = {'func': 'partsearchgetpartgroupsdd',
                'mfr': input_.loc[a, 'Manufacturer'],
                'giveback': json.dumps({'id': 'partsearch_007'}),
                'api_json_request': '1'}

        try:
            b = 0
            while True:

                while True:
                    b += 1
                    try:
                        resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                             data=data,
                                             headers=get_header(),
                                             proxies=get_proxy(),
                                             timeout=(10, 10))
                        break
                    except:
                        continue

                if resp.status_code == 200:
                    json_ = resp.json()
                    if 'redirect_to_url' in json_:
                        continue
                    elif json_['giveback']['id'] == 'partsearch_007' and 'collected_javascript' in json_:
                        break

            # = = = = = = = = = = = = = = =

            from bs4 import BeautifulSoup
            from lxml import etree

            soup = BeautifulSoup(json_['html_replace_sections']['ddrepl[partgroup_][container]'], 'lxml')
            html = etree.HTML(str(soup))

            # = = = = = = = = = = = = = = =

            list_group = html.xpath('//select[@id="partgroup_"]/option/@value')[1:]

            # = = = = = = = = = = = = = = =

            if not list_group:
                raise

            # = = = = = = = = = = = = = = =

            df_temp = pd.DataFrame({'Manufacturer No': input_.loc[a, 'Manufacturer No'],
                                    'Manufacturer': input_.loc[a, 'Manufacturer'],
                                    'Group No': [i+1 for i in range(len(list_group))],
                                    'Group': list_group})

            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            df_temp = pd.DataFrame([{'Manufacturer No': input_.loc[a, 'Manufacturer No'],
                                     'Manufacturer': input_.loc[a, 'Manufacturer']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {input_.loc[a, 'Manufacturer No']}.{input_.loc[a, 'Manufacturer']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Manufacturer No', 'Group'],
                                                ascending=[True, True],
                                                ignore_index=True)
    output_correct.to_excel(f'''./{param_date}/2.group-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['Manufacturer No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel(f'./{param_date}/group_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：547

[状态：ok，尝试次数：1] - 1.3D
[剩余数量：527] - [当前时间：12:33:26]

[状态：ok，尝试次数：1] - 20.AJUSA
[剩余数量：526] - [当前时间：12:33:26]

[状态：ok，尝试次数：1] - 3.AAE
[剩余数量：525] - [当前时间：12:33:26]

[状态：ok，尝试次数：1] - 15.AIMCO
[剩余数量：524] - [当前时间：12:33:26]

[状态：ok，尝试次数：1] - 6.ACDELCO
[剩余数量：523] - [当前时间：12:33:26]

[状态：ok，尝试次数：1] - 2.3M
[剩余数量：522] - [当前时间：12:33:26]

[状态：ok，尝试次数：1] - 8.ACI
[剩余数量：521] - [当前时间：12:33:26]

[状态：ok，尝试次数：1] - 12.AEM INDUCTION
[剩余数量：520] - [当前时间：12:33:27]

[状态：ok，尝试次数：1] - 16.AIRAID
[剩余数量：519] - [当前时间：12:33:27]

[状态：ok，尝试次数：1] - 14.AGS
[剩余数量：518] - [当前时间：12:33:27]

[状态：ok，尝试次数：1] - 5.ACCEL
[剩余数量：517] - [当前时间：12:33:27]

[状态：ok，尝试次数：1] - 24.ALBANY
[剩余数量：516] - [当前时间：12:33:27]

[状态：ok，尝试次数：1] - 29.AMERICAN COMPONENTS
[剩余数量：515] - [当前时间：12:33:29]

[状态：ok，尝试次数：1] - 18.AIRTEX
[剩余数量：514] - [当前时间：12:33:29]

[状态：ok，尝试次数：1] - 33.ANCO
[剩余数量：513] - [当前时间：12:33:30]

[状态：ok，尝试次数：1] - 30.AMERICAN REMANUFACTURERS INC.
[剩余数量：512] - [当前时间：12:33:30]

[状态：ok，尝试次数：1] - 25.ALLIGATOR
[剩余数量：511] - [当前时间：12:33:30]

[状态